In [2]:
# Librerías generales

import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

### Lectura de datos

In [3]:
import os
import numpy as np
import pickle

# Define la carpeta de datos (ajusta esto según el nombre que reciban los datos en Kaggle)
data_folder_name = '../input/dataselected'  

# Definir una función para obtener los nombres de archivos en una carpeta
def getFileNames(folder_name):
    file_names = os.listdir(folder_name)
    return file_names

print(getFileNames(data_folder_name))

# Lectura de archivos '.npy'
x_test = np.load(f'{data_folder_name}/x_sub_val.npy')
x_train = np.load(f'{data_folder_name}/x_sub_train_aug.npy')

y_train_coarse = np.load(f'{data_folder_name}/y_sub_train_aug_1.npy')
y_train_fine = np.load(f'{data_folder_name}/y_sub_train_aug_0.npy')

y_test_coarse = np.load(f'{data_folder_name}/y_sub_val_1.npy')
y_test_fine = np.load(f'{data_folder_name}/y_sub_val_0.npy')

# Lectura de archivos '.pck'
with open('/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck', "rb") as f:
    coarse_label_names = pickle.load(f)

with open('/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck', "rb") as f:
    fine_label_names = pickle.load(f)

# Información de las dimensiones de los datos
print('Dimensiones de los datos:')
print(x_test.shape)
print(x_train.shape)
print(y_train_coarse.shape)
print(y_train_fine.shape)

print('Cantidad de clases:')
print(np.shape(coarse_label_names))
print(np.shape(fine_label_names))


['x_sub_val.npy', 'y_sub_train_aug_0.npy', 'y_sub_train_aug_1.npy', 'y_sub_val_1.npy', 'x_sub_train_aug.npy', 'y_sub_val_0.npy']
Dimensiones de los datos:
(10000, 32, 32, 3)
(1040000, 32, 32, 3)
(1040000,)
(1040000,)
Cantidad de clases:
(20,)
(100,)


In [5]:
from keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout, Flatten, LayerNormalization,BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Model
from keras.regularizers import l2

### MODELO 1

In [6]:
#OTRA ESTRUCTURA
# Entrada y normalización
input_layer = Input(shape=(32, 32, 3), name="matrix_input")

# Estructura de la red
x = BatchNormalization(axis=-1, name="normalization_layer")(input_layer)
flatten_layer = Flatten(name="flattened_input")(x)

# Ajustes de las capas ocultas y Dropout
dense_1024 = Dense(1024, activation='relu', name="dense_1024")(flatten_layer)
dropout_1024 = Dropout(0.3)(dense_1024)
dense_512 = Dense(512, activation='relu', name="dense_512")(dropout_1024)
dropout_512 = Dropout(0.2)(dense_512)
dense_256 = Dense(256, activation='relu', name="dense_256")(dropout_512)
dropout_256 = Dropout(0.2)(dense_256)
dense_128 = Dense(128, activation='relu', name="dense_128")(dropout_256)
dense_64 = Dense(64, activation='relu', name="dense_64")(dense_128)

# Salidas
fine_output = Dense(100, activation='softmax', name='fine_output')(dense_64)

# Fine-grain prediction branch (100 classes)
fine_output = Dense(100,
                           activation='softmax',
                           name='fine_output')(dense_64)

# Coarse-grain prediction branch (20 classes)
coarse_output = Dense(20,
                             activation='softmax',
                             name='coarse_output')(dropout_512)

# Defino el modelo con dos salidas
myModel = Model(
    inputs                  = input_layer,
    #outputs                 = [fine_output, coarse_output]
    outputs                 = [fine_output]
)
"""myModel2 = Model(
    inputs                  = input_layer,
    outputs                 = [coarse_output]
)"""

# Print model summary
myModel.summary()

#myModel2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ matrix_input (InputLayer)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_layer             │ (None, 32, 32, 3)      │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flattened_input (Flatten)       │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1024 (Dense)              │ (None, 1024)           │     3,146,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_512 (Dense)               │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_256 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_128 (Dense)               │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fine_output (Dense)             │ (None, 100)            │         6,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,850,544 (14.69 MB)

 Trainable params: 3,850,538 (14.69 MB)

 Non-trainable params: 6 (24.00 B)

In [7]:
# Shuffle de los datos
indexes_train   = np.arange(len(x_train))
indexes_test    = np.arange(len(x_test))

np.random.shuffle(indexes_train)
np.random.shuffle(indexes_test)

# Shuffle de datos de train

x_train_dup         = x_train[indexes_train]
y_train_coarse_dup  = y_train_coarse[indexes_train]
y_train_fine_dup    = y_train_fine[indexes_train]

# Shuffle de datos de test
x_test_dup         = x_test[indexes_test]
y_test_coarse_dup  = y_test_coarse[indexes_test]
y_test_fine_dup    = y_test_fine[indexes_test]

## CALLBACKS


In [8]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard, LambdaCallback
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [9]:
# Learning rate decay
initial_learning_rate = 1e-4
decay_steps = 4063  # número de pasos ajustado al dataset, podría ser un número mayor si el learning rate baja muy rápido
decay_rate = 0.96   # una tasa de decay ligeramente menor para un ajuste más gradual

lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

In [10]:


rlrop = ReduceLROnPlateau(
    monitor = "val_accuracy",
    factor = 0.2,
    patience = 5,
    verbose = 1,
    min_lr = 1e-6
)


rlrop2 = ReduceLROnPlateau(
    monitor = "accuracy",
    factor = 0.2,
    patience = 5,
    verbose = 1,
    min_lr = 1e-6
)

es = EarlyStopping(
    monitor = "val_accuracy",
    patience = 8,
    verbose = 1,
    restore_best_weights = True
)

es2 = EarlyStopping(
    monitor = "accuracy",
    patience = 10,
    verbose = 1,
    restore_best_weights = True
)

mc2 = ModelCheckpoint(
    "best_weights_v1.weights.h5",
    monitor = "accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,

)


mc = ModelCheckpoint(
    "best_weights_v2.weights.h5",
    monitor = "val_accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,

)



tb = TensorBoard(
    log_dir = "logs"
)

### MODELO 1 COMPILE Y FIT

In [11]:
myModel.compile(
    optimizer               = Adam(learning_rate=initial_learning_rate),
    #loss                    = {'fine_output': 'sparse_categorical_crossentropy', 'coarse_output': 'sparse_categorical_crossentropy'},
    loss                    = {'fine_output': 'sparse_categorical_crossentropy'},
    loss_weights            = None,
    #metrics                 = {'fine_output': 'categorical_accuracy', 'coarse_output': 'categorical_accuracy'},
    metrics                 = {'fine_output': 'accuracy'},
    weighted_metrics        = None,
    run_eagerly             = False,
    steps_per_execution     = 1,
    jit_compile             = "auto",
    auto_scale_loss         = True,
)

In [12]:
#FIT PRUEBA
history = myModel.fit(
    x                       = x_train_dup,
    y                       = {'fine_output': y_train_fine_dup},
    batch_size              = 256,
    epochs                  = 50,
    verbose                 = "auto",
    callbacks               = [LambdaCallback(on_epoch_end=lambda epoch, logs: print(logs if logs is not None else "No logs available")),rlrop,es,mc2,mc],
    validation_split        = 0.0,
    validation_data         = (x_test_dup, {'fine_output': y_test_fine_dup}),
    shuffle                 = True,
    class_weight            = None,
    sample_weight           = None,
    initial_epoch           = 0,
    steps_per_epoch         = None,
    validation_steps        = None,
    validation_batch_size   = None,
    validation_freq         = 1,
)

Epoch 1/50


I0000 00:00:1730136689.126677     100 service.cc:145] XLA service 0x7eb74c004f20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730136689.126722     100 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730136689.126726     100 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  27/4063 ━━━━━━━━━━━━━━━━━━━━ 25s 6ms/step - accuracy: 0.0094 - loss: 4.6858  

I0000 00:00:1730136695.934316     100 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_1', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1730136695.943977     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4063/4063 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0455 - loss: 4.3340

I0000 00:00:1730136718.421589      99 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_1', 16 bytes spill stores, 16 bytes spill loads



{'accuracy': 0.06500384956598282, 'loss': 4.1670331954956055, 'val_accuracy': 0.12349999696016312, 'val_loss': 3.7529399394989014}

Epoch 1: accuracy improved from -inf to 0.06500, saving model to best_weights_v1.weights.h5

Epoch 1: val_accuracy improved from -inf to 0.12350, saving model to best_weights_v2.weights.h5
4063/4063 ━━━━━━━━━━━━━━━━━━━━ 33s 6ms/step - accuracy: 0.0455 - loss: 4.3340 - val_accuracy: 0.1235 - val_loss: 3.7529 - learning_rate: 1.0000e-04
Epoch 2/50
4059/4063 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0949 - loss: 3.9363{'accuracy': 0.099704809486866, 'loss': 3.904357433319092, 'val_accuracy': 0.14730000495910645, 'val_loss': 3.613696575164795}

Epoch 2: accuracy improved from 0.06500 to 0.09970, saving model to best_weights_v1.weights.h5

Epoch 2: val_accuracy improved from 0.12350 to 0.14730, saving model to best_weights_v2.weights.h5
4063/4063 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.0949 - loss: 3.9363 - val_accuracy: 0.1473 - val_loss: 3.6137 -

### MODELO 2

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l2

In [10]:


#OTRA ESTRUCTURA
# Entrada y normalización
input_layer = Input(shape=(32, 32, 3), name="matrix_input")

# Estructura de la red
x = BatchNormalization(axis=-1, name="normalization_layer")(input_layer)
flatten_layer = Flatten(name="flattened_input")(x)


# Ajustes de las capas ocultas y Dropout con regularización l2
dense_1024 = Dense(1024, activation='relu', kernel_regularizer=l2(0.01), name="dense_1024")(flatten_layer)
dropout_1024 = Dropout(0.3)(dense_1024)
dense_512 = Dense(512, activation='relu', kernel_regularizer=l2(0.01), name="dense_512")(dropout_1024)
dropout_512 = Dropout(0.2)(dense_512)
dense_256 = Dense(256, activation='relu', kernel_regularizer=l2(0.01), name="dense_256")(dropout_512)
dropout_256 = Dropout(0.2)(dense_256)
dense_128 = Dense(128, activation='relu', kernel_regularizer=l2(0.01), name="dense_128")(dropout_256)
dense_64 = Dense(64, activation='relu', name="dense_64")(dense_128)

# Salida
fine_output = Dense(100, activation='softmax', name='fine_output')(dense_64)

# Modelo
myModel = Model(inputs=input_layer, outputs=[fine_output])

# Print model summary
myModel.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ matrix_input (InputLayer)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_layer             │ (None, 32, 32, 3)      │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flattened_input (Flatten)       │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1024 (Dense)              │ (None, 1024)           │     3,146,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_512 (Dense)               │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_256 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_128 (Dense)               │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fine_output (Dense)             │ (None, 100)            │         6,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,850,544 (14.69 MB)

 Trainable params: 3,850,538 (14.69 MB)

 Non-trainable params: 6 (24.00 B)

In [11]:
# Shuffle de los datos
indexes_train   = np.arange(len(x_train))
indexes_test    = np.arange(len(x_test))

np.random.shuffle(indexes_train)
np.random.shuffle(indexes_test)

# Shuffle de datos de train

x_train_dup         = x_train[indexes_train]
y_train_coarse_dup  = y_train_coarse[indexes_train]
y_train_fine_dup    = y_train_fine[indexes_train]

# Shuffle de datos de test
x_test_dup         = x_test[indexes_test]
y_test_coarse_dup  = y_test_coarse[indexes_test]
y_test_fine_dup    = y_test_fine[indexes_test]

In [12]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard, LambdaCallback


In [21]:
# Scheduler para el Learning Rate
initial_learning_rate = 1e-4
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True
)


rlrop = ReduceLROnPlateau(
    monitor = "val_accuracy",
    factor = 0.5,
    patience = 3,
    verbose = 1,
    min_lr = 1e-6
)

es = EarlyStopping(
    monitor = "val_accuracy",
    patience = 10,
    verbose = 1,
    restore_best_weights = True
)


rlrop2 = ReduceLROnPlateau(
    monitor = "accuracy",
    factor = 0.2,
    patience = 5,
    verbose = 1,
    min_lr = 1e-6
)

es2 = EarlyStopping(
    monitor = "accuracy",
    patience = 10,
    verbose = 1,
    restore_best_weights = True
)

mc2 = ModelCheckpoint(
    "best_weights.weights.h5",
    monitor = "accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,

)

mc = ModelCheckpoint(
    "best_weights.weights.h5",
    monitor = "val_accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,

)



tb = TensorBoard(
    log_dir = "logs"
)

In [22]:
myModel.compile(
    optimizer               = Adam(learning_rate=lr_schedule),
    #loss                    = {'fine_output': 'sparse_categorical_crossentropy', 'coarse_output': 'sparse_categorical_crossentropy'},
    loss                    = {'fine_output': 'sparse_categorical_crossentropy'},
    loss_weights            = None,
    #metrics                 = {'fine_output': 'categorical_accuracy', 'coarse_output': 'categorical_accuracy'},
    metrics                 = {'fine_output': 'accuracy'},
    weighted_metrics        = None,
    run_eagerly             = False,
    steps_per_execution     = 1,
    jit_compile             = "auto",
    auto_scale_loss         = True,
)

In [24]:
history2 = myModel.fit(
    x                       = x_train_dup,
    y                       = {'fine_output': y_train_fine_dup},
    batch_size              = 256,
    epochs                  = 50,
    verbose                 = "auto",
    callbacks               = [LambdaCallback(on_epoch_end=lambda epoch, logs: print(logs if logs is not None else "No logs available")),rlrop,es,mc2,mc],
    validation_split        = 0.0,
    validation_data         = (x_test_dup, {'fine_output': y_test_fine_dup}),
    shuffle                 = True,
    class_weight            = None,
    sample_weight           = None,
    initial_epoch           = 0,
    steps_per_epoch         = None,
    validation_steps        = None,
    validation_batch_size   = None,
    validation_freq         = 1,
)

Epoch 1/50


I0000 00:00:1730136103.363549     119 service.cc:145] XLA service 0x7e2a7c8b50a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730136103.363613     119 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730136103.363617     119 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  27/4063 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.0089 - loss: 31.4053

I0000 00:00:1730136109.813902     119 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4063/4063 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0405 - loss: 9.9476{'accuracy': 0.053404808044433594, 'loss': 5.980325222015381, 'val_accuracy': 0.08089999854564667, 'val_loss': 4.147796630859375}

Epoch 1: accuracy improved from -inf to 0.05340, saving model to best_weights.weights.h5

Epoch 1: val_accuracy improved from -inf to 0.08090, saving model to best_weights.weights.h5
4063/4063 ━━━━━━━━━━━━━━━━━━━━ 34s 6ms/step - accuracy: 0.0405 - loss: 9.9466 - val_accuracy: 0.0809 - val_loss: 4.1478 - learning_rate: 1.0000e-04
Epoch 2/50
4051/4063 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0686 - loss: 4.2481{'accuracy': 0.07013557851314545, 'loss': 4.230656147003174, 'val_accuracy': 0.093299999833107, 'val_loss': 4.054381847381592}

Epoch 2: accuracy improved from 0.05340 to 0.07014, saving model to best_weights.weights.h5

Epoch 2: val_accuracy improved from 0.08090 to 0.09330, saving model to best_weights.weights.h5
4063/4063 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.

KeyboardInterrupt: 